In [1]:
import re
import pandas as pd
import numpy as np
import datetime

In [2]:
def parser(file):

    lines = []

    with open(file, 'r') as file:
        for line in file:
            lines.append(line.rstrip('\n'))

    #Extract Date
    Date = (re.split('[\s]{4,}', lines[1].strip()))
    Date = Date[0]
    Date = Date.replace('Run Date: ', '')

    #Extract indices to cut list
    indices = [i for i, s in enumerate(lines) if '------' in s]
    indicest = [i for i, s in enumerate(lines) if 'TOTAL  -' in s]

    #Create list with only well licenses
    df1 = lines[(indices[1]+1):(indicest[0]-1)]

    dfsplit = lines[indices[1]]

    def find(s, ch):
        return [i for i, ltr in enumerate(s) if ltr == ch]

    cut = find(dfsplit,' ')

    df2 = []
    for line in df1:
        df2.append(line[0:cut[0]])
        df2.append(line[cut[0]:cut[1]])
        df2.append(line[cut[1]:cut[2]])
        df2.append(line[cut[2]:cut[3]])
        df2.append(line[cut[3]:cut[4]])
        df2.append(line[cut[4]:cut[5]])
        df2.append(line[cut[5]:cut[6]])
        df2.append(line[cut[6]:cut[7]])
        df2.append(line[cut[7]:cut[8]])
        df2.append(line[cut[8]:cut[9]])
        df2.append(line[cut[9]:cut[10]])
        df2.append(line[cut[10]:])

    #Create a list without all spaces
    df3 = []
    for line in df2:
        df3.extend(re.split('[\s]{5,}', line.strip()))

    df3 = [df3[x:x+12] for x in range(0, len(df3), 12)]

    #create a dataframe out of the list
    if df3 != []:
        data1 = pd.DataFrame(df3)
        data1.set_axis(['WELL ID', 'WELL NAME', 'LICENCE', 'CONTRACTOR BA ID', 'CONTRACTOR NAME', 'RIG NUMBER', 'ACTIVITY DATE', 'FIELD CENTRE', 'BA ID', 'LICENSEE', 'NEW PROJECTED TOTAL DEPTH', 'ACTIVITY TYPE'], axis = 1, inplace=True)
        data1['DATE'] = Date

        data1['ATS'] = data1['WELL ID'].str.replace(r'[^0-9]+', '')
        data1['ATS'] = data1['ATS'].str[-9:-1]
        data1['ATS'] = data1['ATS'].astype('int64')
        
        data1['DATE'] = data1['DATE'].astype('datetime64')
        data1['WEEK'] = data1['DATE'].dt.week
        data1['MONTH'] = data1['DATE'].dt.month
        data1['YEAR'] = data1['DATE'].dt.year

        convcomp = pd.read_csv('ConvComp.csv')
        convcomp['ConvDF'] = convcomp['ConvDF'].astype('int64')
        dfm = pd.merge(data1, convcomp, how='left', left_on='ATS', right_on='ConvDF')

    return dfm

In [3]:
import os

from pathlib import Path

directory_in_str = "Data/ST49Hist/"

directory = os.fsencode(directory_in_str) 

columns = ['WELL ID', 'WELL NAME', 'LICENCE', 'CONTRACTOR BA ID', 'CONTRACTOR NAME', 'RIG NUMBER', 'ACTIVITY DATE', 'FIELD CENTRE', 'BA ID', 'LICENSEE', 'NEW PROJECTED TOTAL DEPTH', 'ACTIVITY TYPE', 'DATE', 'ATS', 'ConvDF', 'Lat', 'Long', 'WEEK', 'MONTH']

df = pd.DataFrame(columns=columns)

ext = ['.txt','.TXT']

for file in os.listdir(directory): 
    filename = os.fsdecode(file) 
    if filename.endswith(tuple(ext)):
        print(filename) 
        data = parser(os.path.join(directory_in_str, filename))
        df = df.append(data, ignore_index=True, sort=False)
        continue 
    else: 
        continue

516.txt
25165.txt
29767.txt
2875.txt
28395.txt
7312.txt
9286.txt
12936.txt
19201.txt
13946.txt
30115.txt
6405.txt
4497.txt
9701.txt
20090.txt
20943.txt
15009.txt
14391.txt
6964.txt
21516.txt
30370.txt
16643.txt
24237.txt
23095.txt
18440.txt
28585.txt
12825.txt
16242.txt
2063.txt
13387.txt
16427.txt
5537.txt
27786.txt
26032.txt
13218.txt
13588.txt
1322.txt
12752.txt
14447.txt
3193.txt
13766.txt
4463.txt
17106.txt
220.txt
10452.txt
6860.txt
16329.txt
11669.txt
9785.txt
6491.txt
6939.txt
12125.txt
4395.txt
7551.txt
11757.txt
3569.txt
25337.txt
17917.txt
18560.txt
14222.txt
1439.txt
9387.txt
17171.txt
1399.txt
15821.txt
20071.txt
28417.txt
7809.txt
31824.txt
31177.txt
29223.txt
9279.txt
435.txt
31719.txt
16950.txt
22038.txt
6363.txt
16520.txt
21199.txt
5801.txt
19864.txt
30119.txt
16606.txt
20851.txt
12468.txt
12879.txt
5365.txt
28392.txt
5509.txt
8885.txt
25784.txt
31200.txt
17207.txt
32177.txt
15486.txt
21875.txt
29235.txt
11644.txt
25852.txt
7104.txt
22640.txt
23887.txt
SPUD0712.txt
228

In [4]:
df

,WELL ID,WELL NAME,LICENCE,CONTRACTOR BA ID,CONTRACTOR NAME,RIG NUMBER,ACTIVITY DATE,FIELD CENTRE,BA ID,LICENSEE,NEW PROJECTED TOTAL DEPTH,ACTIVITY TYPE,DATE,ATS,ConvDF,Lat,Long,WEEK,MONTH,YEAR
0,03/16-25-031-25W4/0,ARTIS HZ 103 TWINING 16-25-31-25,0487843,0WA4,Trinidad Drilling Ltd.,418,20 Feb 2018 11:00:00 PM,Red Deer,A77W,Artis Exploration Ltd.,,Drill To LD,2018-02-21,25031254,25031254,51.685278,-113.406654,8,2,2018.0
1,02/15-07-035-04W4/0,KARVE 102 HZ PROVOST 15-7-35-4,0487967,0Z41,Ensign Drilling Inc.,348,20 Feb 2018 10:00:00 PM,Wainwright,A7BT,Karve Energy Inc.,,Drill To LD,2018-02-21,7035044,7035044,51.990032,-110.552747,8,2,2018.0
2,00/02-02-037-01W5/0,VESTA TINDA 2-2-37-1,0488091,A6EC,Twilight Drilling Ltd.,1,21 Feb 2018 05:15:00 AM,Red Deer,A0L1,Vesta Energy Ltd.,,Drill To LD,2018-02-21,2037015,2037015,52.151789,-114.025698,8,2,2018.0
3,03/11-24-041-02W4/0,ROCKEAST HZ 103 HAYTER 11-24-41-2,0484986,A6NZ,Lasso Drilling Corporation,3,20 Feb 2018 09:30:00 PM,Wainwright,A6M1,Rockeast Energy Corp.,,Drill To LD,2018-02-21,24041024,24041024,52.543120,-110.149044,8,2,2018.0
4,02/01-10-043-08W5/0,OBE 102 DD WILLGR 1-10-43-8,0484674,A6PR,Nabors Drilling Canada Limited,26,21 Feb 2018 04:00:00 PM,Red Deer,0BP8,Obsidian Energy Ltd.,,Drill To LD,2018-02-21,10043085,10043085,52.691222,-115.064928,8,2,2018.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31180,15/04-31-086-09W4/0,ATH AE05P HANGST 4-31-86-9,0455858,0Y52,Precision Drilling Corporation,192,14 Mar 2014 04:30:00 PM,Bonnyville,A28L,Athabasca Oil Corporation,1333,Resumption,2014-03-15,31086094,31086094,56.500773,-111.401904,11,3,2014.0
31181,05/05-23-090-14W4/0,MACOP AC01P ELLS 5-23-90-14,0461627,0Y52,Precision Drilling Corporation,144,14 Mar 2014 12:45:00 PM,Bonnyville,A5LG,Mackay Operating Corp.,,Resumption,2014-03-15,23090144,23090144,56.820615,-112.105523,11,3,2014.0
31182,AB/15-10-091-10W4/0,SUNCOR VOYAGEUR 13-003B 15-10-91-10,0465744,0Y80,SDS Drilling,38,15 Mar 2014 01:30:00 AM,Bonnyville,0054,Suncor Energy Inc.,,Drill To LD,2014-03-15,10091104,10091104,56.879014,-111.508184,11,3,2014.0
31183,04/14-34-108-08W6/0,HUSKY HZ C1-14 RAINBOW 14-34-108-8,0465220,0Z41,Ensign Drilling Inc.,110,14 Mar 2014 07:15:00 PM,High Level,0R46,Husky Oil Operations Limited,,Drill To LD,2014-03-15,34108086,34108086,58.419841,-119.227988,11,3,2014.0


Export to CSV

In [5]:
export = df.to_csv (r'ST49 Historical.csv', index = False)

dataframe = pd.read_csv('Historical.csv')
dftotal = pd.merge(df, dataframe, how='left', left_on='LICENCE', right_on='LicenceNumber')
#dftotal.dropna(inplace=True)
dftotal

dftotal.to_csv('ST1_ST49.csv', index=False)

import plotly.express as px

df["LICENSEE"] = df["LICENSEE"].astype(str)

fig = px.scatter_geo(df, lon = 'Long', lat = 'Lat', text = 'LICENSEE', color = 'LICENSEE')

fig.update_geos(visible=False, resolution=50, showcountries=True, showsubunits=True, scope="north america", fitbounds='locations')

fig.update_layout(height=1000, width=1000, margin={"r":0,"t":0,"l":0,"b":0})

fig.show()
